In [1]:
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests
import acquire as a
import re
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize.toktok import ToktokTokenizer
import requests
import unicodedata
import nltk
from textblob import TextBlob, Word
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.stem import WordNetLemmatizer
import prepare as p
import acquire as a
import new_lib as nl
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = p.get_links('links2.csv')

In [3]:
df2 = a.get_dataframe(df.href)

In [4]:
df2 = p.get_languages(df2)

In [5]:
df2 = p.clean_languages(df2)

In [6]:
df2 = p.drop_blanks(df2)

In [7]:
df2['readme_contents'] = df2['readme_contents'].apply(p.basic_clean)

In [8]:
words = ['java', 'blackjack','card','ace','rich', 'poor', 
         'bust', 'split', 'variables', 'loop', 'function', 'syntax', 'library', 'conditional', 'frameworks']

In [9]:
df2['readme_contents'] = df2['readme_contents'].apply(p.remove_stopwords, extra_words = words)

In [10]:
df2['readme_contents'] = df2['readme_contents'].apply(p.tokenized)

In [11]:
df2['readme_contents'] = df2['readme_contents'].apply(p.split)

In [12]:
df2 = df2.dropna()

In [13]:
df2 = df2[df2['language'] != 'Other']

In [14]:
train, val, test, X_train, y_train, X_val, y_val, X_test, y_test = nl.train_vailidate_test_split(df2, 'language')

In [15]:
len(train)

191

In [16]:
len(val)

82

In [17]:
len(test)

69

In [18]:
train.isna().sum()

repo               0
language           0
readme_contents    0
dtype: int64

In [19]:
train

,repo,language,readme_contents
311,learn-co-students/cli-applications-simple-blac...,Ruby,simplifi procedur learn goal util logic loop g...
406,grapeJUICE1/python-mini-projects,Python,python mini project repositori contain mini py...
225,learn-co-students/simple-blackjack-cli-chicago...,Ruby,simplifi procedur learn goal util logic loop g...
361,guilhermedelemos/blackjack,Java,vinteeum build statushttpstravisciorgguilherme...
53,tedyoung/blackjack-r2ha-2022,Java,refactor hexagon architectur starter code base...
...,...,...,...
437,DEEBAZZ890/Blackjack-game,Python,21blackjackgam singl deck card pick two player...
286,nehctuk/teampun-ther,Java,teampunth
215,learn-co-students/simple-blackjack-cli-chicago...,Ruby,simplifi procedur learn goal util logic loop g...
61,learn-co-students/simple-blackjack-cli-dumbo-w...,Ruby,simplifi procedur learn goal util logic loop g...


In [20]:
word_list = []
for n in train['readme_contents']:
    word_list.extend(n.split(' '))

In [21]:
series = pd.Series(word_list)

In [22]:
series.value_counts().head(20)

method     9585
test       2444
use        2234
line       2054
put        2031
command    1957
user       1954
input      1917
call       1847
player     1817
file       1762
program    1620
output     1581
runner     1558
hit        1422
game       1392
total      1338
rubi       1336
card       1269
run        1260
dtype: int64

In [23]:
tfidf = TfidfVectorizer()

In [24]:
X_train = tfidf.fit_transform(X_train['readme_contents'])

In [25]:
X_val = tfidf.transform(X_val['readme_contents'])

In [26]:
X_val = pd.DataFrame(X_val.todense())

In [27]:
clf = DecisionTreeClassifier(max_depth=6, random_state=77)

In [28]:
X_train = pd.DataFrame(X_train.todense())

In [29]:
clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=6, random_state=77)

In [30]:
y_pred = clf.predict(X_train)

In [31]:
y_pred_proba = clf.predict_proba(X_train)

In [32]:
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
      .format(clf.score(X_train, y_train)))

Accuracy of Decision Tree classifier on training set: 0.92


In [33]:
confusion_matrix(y_train, y_pred)

array([[  8,   5,   0,   0,   0],
       [  0,  22,   0,   0,   0],
       [  0,   6,  13,   0,   0],
       [  0,   3,   0,  18,   0],
       [  0,   2,   0,   0, 114]])

In [34]:
ConfusionMatrixDisplay(confusion_matrix(y_train, y_pred))

In [35]:
pd.crosstab(y_train, y_pred)

col_0,C_based,Java,JavaScript,Python,Ruby
language,,,,,
C_based,8,5,0,0,0
Java,0,22,0,0,0
JavaScript,0,6,13,0,0
Python,0,3,0,18,0
Ruby,0,2,0,0,114


In [36]:
val_pred = clf.predict(X_val)

In [37]:
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
      .format(clf.score(X_val, y_val)))

Accuracy of Decision Tree classifier on training set: 0.87


In [38]:
rf = RandomForestClassifier(bootstrap=True, 
                            class_weight=None, 
                            criterion='gini',
                            min_samples_leaf=5,
                            n_estimators=250,
                            max_depth=6, 
                            random_state=77)

In [39]:
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=6, min_samples_leaf=5, n_estimators=250,
                       random_state=77)

In [40]:
rf_pred = rf.predict(X_train)

In [41]:
print('Accuracy of random forest classifier on training set: {:.2f}'
     .format(rf.score(X_train, y_train)))

Accuracy of random forest classifier on training set: 0.87


In [42]:
rf2_pred = rf.predict(X_val)

In [43]:
print('Accuracy of random forest classifier on training set: {:.2f}'
     .format(rf.score(X_val, y_val)))

Accuracy of random forest classifier on training set: 0.84


In [44]:
df2.language.value_counts()

Ruby          213
Python         36
Java           36
JavaScript     35
C_based        22
Name: language, dtype: int64

In [45]:
len(df2[df2['language'] == 'Ruby'])/len(df2)

0.6228070175438597

In [53]:
X_test = tfidf.transform(X_test['readme_contents'])

In [54]:
X_test = pd.DataFrame(X_test.todense())

In [55]:
test_pred = clf.predict(X_test)

In [56]:
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
      .format(clf.score(X_test, y_test)))

Accuracy of Decision Tree classifier on training set: 0.77


In [57]:
pd.crosstab(y_test, test_pred)

col_0,C_based,Java,JavaScript,Python,Ruby
language,,,,,
C_based,0,5,0,0,0
Java,1,8,0,0,0
JavaScript,0,8,3,0,0
Python,0,2,0,2,0
Ruby,0,0,0,0,40


In [58]:
test_preds = rf.predict(X_test)

In [59]:
print('Accuracy of random forest classifier on training set: {:.2f}'
     .format(rf.score(X_test, y_test)))

Accuracy of random forest classifier on training set: 0.74
